In [1]:
# data wrangling tools
import pandas as pd
import numpy as np

# visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# statistical analysis
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# datadotworld SDK
import datadotworld as ddw
#% load_ext watermark

import time

/home/jrfox9/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# retrieve the cancer data from data.world
mortdf = ddw.query('johnfox/cancer-analysis-hackathon-challenge', 
                   'SELECT * FROM death').dataframe
incddf = ddw.query('johnfox/cancer-analysis-hackathon-challenge', 
                   'SELECT * FROM incd').dataframe

#Eliminating FIPS with null value
mortdf = mortdf[mortdf.fips.notnull()]
incddf = incddf[incddf.fips.notnull()]

#Here we make sure all entries for FIPS are 5 digits long and pad with 0
mortdf['fips'] = mortdf.fips.apply(lambda x: str(int(x))).astype(np.object_)\
                            .str.pad(5, 'left','0')
incddf['fips'] = incddf.fips.apply(lambda x: str(int(x))).astype(np.object_)\
                            .str.pad(5, 'left','0')

#We drop unnecessary columns
mortdf.drop(mortdf.columns[[0,2,4,5,7,8,9,10]], axis = 1, inplace = True)
incddf.drop(incddf.columns[[0,2,4,5,7,8,9,10]], axis = 1, inplace = True)

#We rename the columns to our liking
mortdf.rename(columns = {mortdf.columns[0]:'FIPS', mortdf.columns[1]:'Mortality_Rate',
                        mortdf.columns[2]:'Avg_Ann_Deaths'}, inplace = True)
incddf.rename(columns = {incddf.columns[0]:'FIPS',incddf.columns[1]:'Incidence_Rate',
                        incddf.columns[2]:'Avg_Ann_Incidence'}, inplace = True)

The data for regressor variables will be obtained from  
https://data.world/uscensusbureau/2014-american-community-survey
We will use as regressors
* Poverty Data
* Income Data (various catagories within income)


We are also thinking of including indicator variables which take into account the regions of the United States

* West
* Midwest
* Northeast
* South

Since the cancer data is divided into counties, we must gather the data individually from each state; due to the fact that the data for each particular state is also divided into counties. Unlike the data of the entire United States which is divided at the smallest level into states. 

In [3]:
#Retrieve a list of table names (by state)
pov = ddw.load_dataset('uscensusbureau/acs-2015-5-e-poverty', force_update=True)

tables = []
for i in pov.tables:
    if len(i) == 2:
        tables.append(i)

#remove Puerto Rico
tables.remove('pr')

In [4]:
#print(len(tables))
#np.array(tables)
mortdf.head()

,FIPS,Mortality_Rate,Avg_Ann_Deaths
0,00000,44.7,156865
1,02185,132.5,5
2,21095,100.3,36
3,28039,78.6,19
4,40001,56.1,14


**Poverty Data**

We do not need all the information in the tables we must only query the following columns

* State
* StateFIPS
* CountyFIPS
* AreaName
* B17001_002--> POVERTY STATUS IN THE PAST 12 MONTHS BY SEX BY AGE for Population For Whom Poverty Status Is Determined% Income in the past 12 months below poverty level:

All the information in the rows isn't necessary. The information for each particular county is located where the SummaryLevel=50

In [5]:
#Retrieve the Census poverty data from data.world
start = time.time()

#a string - the poverty columns we want from the Census ACS
cols = '`State`,`StateFIPS`,`CountyFIPS`,`AreaName`,`B17001_002`'

#Call the data for each state and concatenate
for i,state in enumerate(tables):
    if i == 0:
        povdf = ddw.query('uscensusbureau/acs-2015-5-e-poverty',
                  '''SELECT %s FROM `AK`
                     WHERE SummaryLevel=50''' % cols).dataframe 
    else:
        df = ddw.query('uscensusbureau/acs-2015-5-e-poverty',
                       '''SELECT %s FROM `%s`
                          WHERE SummaryLevel=50''' % (cols, state.upper())).dataframe
    
        povdf = pd.concat([povdf, df], ignore_index=True)
    
end = time.time()

print(end-start)

#Add leading zeros to the state and county FIPS codes
povdf['StateFIPS'] = povdf.StateFIPS.astype(np.object_)\
                                    .apply(lambda x: str(x))\
                                    .str.pad(2, 'left', '0')
povdf['CountyFIPS'] = povdf.CountyFIPS.astype(np.object_)\
                                    .apply(lambda x: str(x))\
                                    .str.pad(3, 'left', '0')

povdf.rename(columns={'B17001_002':'All_Poverty'}, inplace=True)

154.98529481887817


In [6]:
povdf.head()

,State,StateFIPS,CountyFIPS,AreaName,All_Poverty
0,AK,02,013,"Aleutians East Borough, Alaska",553
1,AK,02,016,"Aleutians West Census Area, Alaska",499
2,AK,02,020,"Anchorage Municipality, Alaska",23914
3,AK,02,050,"Bethel Census Area, Alaska",4364
4,AK,02,060,"Bristol Bay Borough, Alaska",69


The following columns will be analyzed  regarding **Income**

B19054_002	INTEREST, DIVIDENDS, OR NET RENTAL INCOME IN THE PAST 12 MONTHS FOR HOUSEHOLDS for Households% With interest, dividends, or net rental income	Income

B19055_002	SOCIAL SECURITY INCOME IN THE PAST 12 MONTHS FOR HOUSEHOLDS for Households% With Social Security income	Income

B19056_002	SUPPLEMENTAL SECURITY INCOME (SSI) IN THE PAST 12 MONTHS FOR HOUSEHOLDS for Households% With Supplemental Security Income (SSI)	Income

B19057_002	PUBLIC ASSISTANCE INCOME IN THE PAST 12 MONTHS FOR HOUSEHOLDS for Households% With public assistance income	Income

B19058_002	PUBLIC ASSISTANCE INCOME OR FOOD STAMPS/SNAP IN THE PAST 12 MONTHS FOR HOUSEHOLDS for Households% With cash public assistance or Food Stamps/SNAP	Income

B19059_002	RETIREMENT INCOME IN THE PAST 12 MONTHS FOR HOUSEHOLDS for Households% With retirement income	Income


In [7]:
cols = '`StateFIPS`,`CountyFIPS`,'\
        '`B19054_002`, `B19055_002`, `B19056_002`,`B19057_002`,`B19058_002`,`B19059_002`'

start = time.time()

for i, state in enumerate(tables):
    if i == 0:
        incomedf = ddw.query('uscensusbureau/acs-2015-5-e-income',
                             '''SELECT %s FROM `AK`
                             WHERE SummaryLevel=50'''% cols).dataframe
    else:
        df = ddw.query('uscensusbureau/acs-2015-5-e-income',
                      '''SELECT %s FROM `%s`
                      WHERE SummaryLevel=50'''%(cols, state.upper())).dataframe
        
        incomedf = pd.concat([incomedf, df], ignore_index=True)
        
        end = time.time()

print(end - start)

incomedf['StateFIPS'] = incomedf.StateFIPS.astype(np.object_)\
                                .apply(lambda x: str(x))\
                                .str.pad(2,'left', '0')
incomedf['CountyFIPS'] = incomedf.CountyFIPS.astype(np.object_)\
                                .apply(lambda x: str(x))\
                                .str.pad(3,'left', '0')
        
incomedf.rename(columns={'B19054_002':'Med_Income_With_Interest_Dividends',
                        'B19055_002':'Median_Income_With_SS','B19056_002':'Median_Income_With_SSI',
                        'B19057_002':'Median_Income_With_Public_Assistance',
                        'B19058_002':'Meadian_Income_With_Public_Assis_Food_Stamps',
                        'B19059_002':'Retirement_Income'}, inplace=True)    

107.89262557029724


In [8]:
incomedf.head()


,StateFIPS,CountyFIPS,Med_Income_With_Interest_Dividends,Median_Income_With_SS,Median_Income_With_SSI,Median_Income_With_Public_Assistance,Meadian_Income_With_Public_Assis_Food_Stamps,Retirement_Income
0,02,013,407,113,15,37,109,42
1,02,016,462,108,25,27,74,66
2,02,020,48585,18089,4620,6076,11579,17805
3,02,050,3036,937,293,694,1888,505
4,02,060,211,59,10,20,36,42


I have decided not to include insurance

## Merge DataFrames

We're going to merge on the FIPS columns. It's a 5 digit code. The first 2 digits are the states code and the las 3 digits are FIPS the county code.

1. First, let's create an "FIPS" in each dataframe by concatenating the state and the county codes.
1. Then, let check to make sure they properly match up.

In [9]:
dfs = [povdf, incomedf, incddf, mortdf]

In [10]:
# create FIPS features
for df in [povdf, incomedf]:
    df['FIPS'] = df.StateFIPS + df.CountyFIPS
    df.drop(['StateFIPS', 'CountyFIPS'], axis=1, inplace=True)


In [11]:
#Check if FIPS column in incomedf is equal to FIPS column in povdf
print(incomedf.FIPS.equals(povdf.FIPS))


True


OK, it looks like the FIPS column in each dataframe is formatted well and they match up. Let's join them together into one large dataframe.

In [12]:
for i, j in enumerate(dfs):
    if i == 0:
        fulldf = j.copy()
    else:
        fulldf = fulldf.merge(j, how='inner', on='FIPS')

# Exploratory analysis (and continued data cleaning)

In [13]:
fulldf.shape
#print(len(fulldf))

(3134, 14)

In [14]:
fulldf.head()

,State,AreaName,All_Poverty,FIPS,Med_Income_With_Interest_Dividends,Median_Income_With_SS,Median_Income_With_SSI,Median_Income_With_Public_Assistance,Meadian_Income_With_Public_Assis_Food_Stamps,Retirement_Income,Incidence_Rate,Avg_Ann_Incidence,Mortality_Rate,Avg_Ann_Deaths
0,AK,"Aleutians East Borough, Alaska",553,02013,407,113,15,37,109,42,*,3 or fewer,*,**
1,AK,"Aleutians West Census Area, Alaska",499,02016,462,108,25,27,74,66,*,3 or fewer,*,**
2,AK,"Anchorage Municipality, Alaska",23914,02020,48585,18089,4620,6076,11579,17805,57,125,42.5,90
3,AK,"Bethel Census Area, Alaska",4364,02050,3036,937,293,694,1888,505,45.7,4,50.2,4
4,AK,"Bristol Bay Borough, Alaska",69,02060,211,59,10,20,36,42,*,3 or fewer,*,**


In [15]:
#check for null values
for col in fulldf.columns:
    print((col, sum(fulldf[col].isnull())))

('State', 0)
('AreaName', 0)
('All_Poverty', 0)
('FIPS', 0)
('Med_Income_With_Interest_Dividends', 0)
('Median_Income_With_SS', 0)
('Median_Income_With_SSI', 0)
('Median_Income_With_Public_Assistance', 0)
('Meadian_Income_With_Public_Assis_Food_Stamps', 0)
('Retirement_Income', 0)
('Incidence_Rate', 0)
('Avg_Ann_Incidence', 0)
('Mortality_Rate', 0)
('Avg_Ann_Deaths', 0)


Creating a data dictionary

In [16]:
data_dict = pd.DataFrame(fulldf.columns.values, index=range(len(fulldf.columns)), columns=['Feture'])

data_dict['Definition'] = ['','', 'People below poverty', 'State + County FIPS', 'Med_Income from invested money',
                          'Med_Income with Social Security', 'Medi_Income with Supplemental Social Security',
                          'Med_Income with Welfare', 'Med_Income with Food Stamps', 'Med_Income with Retirement Income',
                          'Lung Cancer Incedence Rate (per 100,000)', 'Average Lung Cancer Incidence Rate',
                          'Lung Cancer Mortality Rate (per 100,000)','Average Lung Cancer Mortality']

data_dict['Notes'] = ''
data_dict.loc[[10,12], 'Notes'] = "'*' = fewer than 16 reported cases"
data_dict

,Feture,Definition,Notes
0,State,,
1,AreaName,,
2,All_Poverty,People below poverty,
3,FIPS,State + County FIPS,
4,Med_Income_With_Interest_Dividends,Med_Income from invested money,
5,Median_Income_With_SS,Med_Income with Social Security,
6,Median_Income_With_SSI,Medi_Income with Supplemental Social Security,
7,Median_Income_With_Public_Assistance,Med_Income with Welfare,
8,Meadian_Income_With_Public_Assis_Food_Stamps,Med_Income with Food Stamps,
9,Retirement_Income,Med_Income with Retirement Income,


We now need to make sure that all numeric features are actual numeric values and not strings for example.

In [17]:
def get_types(col_name):
    ts = (pd.Series([type(i) for i in fulldf[col_name]]).value_counts())
    print("%s\n" % feature, ts, "\n", "-"*30)

for feature in fulldf.columns:
    get_types(feature)

State
 <class 'str'>    3134
dtype: int64 
 ------------------------------
AreaName
 <class 'str'>    3134
dtype: int64 
 ------------------------------
All_Poverty
 <class 'numpy.int64'>    3134
dtype: int64 
 ------------------------------
FIPS
 <class 'str'>    3134
dtype: int64 
 ------------------------------
Med_Income_With_Interest_Dividends
 <class 'numpy.int64'>    3134
dtype: int64 
 ------------------------------
Median_Income_With_SS
 <class 'numpy.int64'>    3134
dtype: int64 
 ------------------------------
Median_Income_With_SSI
 <class 'numpy.int64'>    3134
dtype: int64 
 ------------------------------
Median_Income_With_Public_Assistance
 <class 'numpy.int64'>    3134
dtype: int64 
 ------------------------------
Meadian_Income_With_Public_Assis_Food_Stamps
 <class 'numpy.int64'>    3134
dtype: int64 
 ------------------------------
Retirement_Income
 <class 'numpy.int64'>    3134
dtype: int64 
 ------------------------------
Incidence_Rate
 <class 'str'>    3134
dtyp

**Columns that seem to need to be fixed: Incidence_Rate, Ave_Ann_Incidence, Mortality_Rate, Avg_Ann_Deaths**

In [18]:
#This script converts values to numeric
def f(columns):
    types = []
    for _, j in enumerate(columns):
        try:
            pd.to_numeric(j)
            
        except:
            types.append(j)
    print(pd.Series(types).value_counts())

f(fulldf.Mortality_Rate)

*    334
dtype: int64


The paper that i'm imitating has proven than the counties with '*' have small populations and they decided to eliminate them, which is something that i will do as well for Mortality_Rate

In [19]:
fulldf = fulldf[fulldf.Mortality_Rate != '*']

## We still need to clean up Incidence_Rate, Ave_Ann_Incidence, and Avg_Ann_Deaths

**For the projects sake i will use as a response variable Mortality and eliminate Incidence_Rate, Avg_Ann_Incidence, and Aveg_Ann_Deaths. Just because some may seem colinear and the other has problems just like the document that I am imitating proposes**

Now I will define the regresor matrix and the response vector 

Population estimates for nomalization

In [20]:
populationdf = ddw.query('nrippner/us-population-estimates-2015',
                         '''SELECT `POPESTIMATE2015`, `STATE`, `COUNTY`
                            FROM `CO-EST2015-alldata`''').dataframe
state = populationdf.STATE.apply(lambda x: str(x))\
                          .str.pad(2, 'left', '0')
county = populationdf.COUNTY.apply(lambda x: str(x))\
                            .str.pad(3, 'left', '0')

populationdf['FIPS'] = state + county

fulldf = fulldf.merge(populationdf[['FIPS', 'POPESTIMATE2015']], on='FIPS', how='inner')


In [21]:
y = pd.to_numeric(fulldf.Mortality_Rate).values
X = fulldf.loc[:,['State', 'Mortality_Rate', 'All_Poverty', 'Med_Income_With_Interest_Dividends', 'Median_Income_With_SS', 'Median_Income_With_SSI', 
                  'Median_Income_With_Public_Assistance', 'Meadian_Income_With_Public_Assis_Food_Stamps', 'Retirement_Income','POPESTIMATE2015'] ]

In [22]:
X.head()

,State,Mortality_Rate,All_Poverty,Med_Income_With_Interest_Dividends,Median_Income_With_SS,Median_Income_With_SSI,Median_Income_With_Public_Assistance,Meadian_Income_With_Public_Assis_Food_Stamps,Retirement_Income,POPESTIMATE2015
0,AK,42.5,23914,48585,18089,4620,6076,11579,17805,298695
1,AK,50.2,4364,3036,937,293,694,1888,505,17946
2,AK,48.8,7752,14008,6245,1320,1738,3452,7104,99631
3,AK,41.4,2110,6132,2490,435,688,1263,3888,32756
4,AK,47.4,5558,7356,6298,1060,1334,2423,4500,58059


The mortality rate is per 100,000 population so we must normalize each variable. The PC stands for per capita

In [23]:
for col in ['All_Poverty', 'Med_Income_With_Interest_Dividends', 'Median_Income_With_SS', 'Median_Income_With_SSI', 
                  'Median_Income_With_Public_Assistance', 'Meadian_Income_With_Public_Assis_Food_Stamps', 'Retirement_Income']:
       
    X[col + "_PC"] = X[col] / X.POPESTIMATE2015 * 10**5

In [24]:
X.head()

,State,Mortality_Rate,All_Poverty,Med_Income_With_Interest_Dividends,Median_Income_With_SS,Median_Income_With_SSI,Median_Income_With_Public_Assistance,Meadian_Income_With_Public_Assis_Food_Stamps,Retirement_Income,POPESTIMATE2015,All_Poverty_PC,Med_Income_With_Interest_Dividends_PC,Median_Income_With_SS_PC,Median_Income_With_SSI_PC,Median_Income_With_Public_Assistance_PC,Meadian_Income_With_Public_Assis_Food_Stamps_PC,Retirement_Income_PC
0,AK,42.5,23914,48585,18089,4620,6076,11579,17805,298695,8006.160130,16265.756039,6056.010312,1546.728268,2034.182025,3876.529570,5960.930046
1,AK,50.2,4364,3036,937,293,694,1888,505,17946,24317.396634,16917.418923,5221.219213,1632.675805,3867.157027,10520.450240,2813.997548
2,AK,48.8,7752,14008,6245,1320,1738,3452,7104,99631,7780.710823,14059.880961,6268.129397,1324.888840,1744.436972,3464.785057,7130.310847
3,AK,41.4,2110,6132,2490,435,688,1263,3888,32756,6441.567957,18720.234461,7601.660764,1328.000977,2100.378557,3855.782147,11869.581145
4,AK,47.4,5558,7356,6298,1060,1334,2423,4500,58059,9573.020548,12669.870304,10847.586076,1825.729000,2297.662722,4173.340912,7750.736320


The categorical variables of the regions in the US are created in JMP, by using the state column.

In [43]:
X.to_csv('~/regression_analysis/data.csv')

Empty DataFrame
Columns: [West]
Index: []
